In [ ]:
import gym
import math
import gym_miniworld
from environment.env_utils import get_data
from dataset_utils.multiview_data import MultiViewData

env = gym.make("MiniWorld-PickupObjs-v0")
env.domain_rand = True
env.max_episode_steps = math.inf
agent_fov = 90

steps_per_epoch = 21
epochs = 4762

multiview_data = MultiViewData(episode_duration=steps_per_epoch,
                               data_buffer_size=int(1e5),
                               batch_size=16)

for i in range(epochs):
    _ = env.reset()
    env.agent.cam_fov_y = agent_fov
    obs, top_down_obs = get_data(env)
    multiview_data.append_obs(obs, top_down_obs, new_episode=True)
    for _ in range(steps_per_epoch-1):
        _, reward, done, info = env.step(env.actions.turn_right)
        obs, top_down_obs = get_data(env)
        multiview_data.append_obs(obs, top_down_obs)
    if i % 100 == 0:
        print(f"epoch:{i}")

multiview_data.save(data_folder="../data/")
env.close()